<a href="https://colab.research.google.com/github/Anil951/YouTube-Video-Recommendation-for-Domain-Specific-Content/blob/main/lstm_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*sentiment analysis using LSTM*

In [ ]:
import pandas as pd

# Load your dataset
file_path = r"result.csv"  # Replace with the path to your dataset
df = pd.read_csv(file_path)

In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

#set of custom stop words
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

lemmatizer = nltk.WordNetLemmatizer()

import nltk
nltk.download('wordnet')

def preprocess_text(review):
    review = re.sub(r"http\S+", "", review)             # removing website links
    review = BeautifulSoup(review, 'lxml').get_text()   # removing html tags
    review = decontract(review)                         # decontracting
    review = re.sub("\S*\d\S*", "", review).strip()     # removing the words with numeric digits
    review = re.sub('[^A-Za-z]+', ' ', review)          # removing non-word characters
    review = review.lower()                             # converting to lower case
    review = [word for word in review.split(" ") if not word in stop_words] # removing stop words
    review = [lemmatizer.lemmatize(token, "v") for token in review] #Lemmatization
    review = " ".join(review)
    review.strip()
    return review

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
df['up_text'] = df['text'].apply(lambda x: preprocess_text(x))
df

,text,sentiment,up_text
0,Love you sir!!,1,love sir
1,Another great explanation by Abdul sir. Thank ...,1,another great explanation abdul sir thank much
2,I had no idea what was going on in the first o...,1,no idea go first one one clear everything
3,great job!!!!,1,great job
4,Thank you Sir !!,1,thank sir
...,...,...,...
4492,I got a little confused after watching this vi...,-1,get little confuse watch video exactly mean ba...
4493,I got 18 points... I think most of the wrong a...,-1,get point think wrong answer forget say
4494,Why is 3a in the first block correct? A lingui...,-1,first block correct linguistic major refer lan...
4495,Sometimes I feel like my English has really ad...,-1,sometimes feel like english really advance eve...


In [ ]:
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with 'text' and 'sentiment' columns

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['up_text'].values)
X = tokenizer.texts_to_sequences(df['up_text'].values)
X = pad_sequences(X)

embed_dim = 128
lstm_out = 100

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))  # Use 'sigmoid' for binary classification
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Y = df['sentiment'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


model.fit(X_train, Y_train, epochs=7, verbose=2, validation_data=(X_test, Y_test))


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 234, 128)          256000    
                                                                 
 spatial_dropout1d (Spatial  (None, 234, 128)          0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 1)                 197       
                                                                 
Total params: 510997 (1.95 MB)
Trainable params: 510997 (1.95 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
(3012, 234) (3012,)
(1485, 234) (1485,)
Epoch 1/7
95/95 - 147s - loss: 0.3843 - a

In [ ]:
def predict_sentiment(model, tokenizer, text):
    # Tokenize and pad the input text
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=model.input_shape[1])

    # Make predictions
    probability = model.predict(padded_sequences)[0]

    # Convert probability to sentiment label
    sentiment_label = "positive" if probability > 0.5 else "negative"

    # Convert sentiment label to target value
    target_value = 1 if sentiment_label == "positive" else -1

    return sentiment_label, target_value, probability

# Example usage:
new_text = "this is a bad video"
sentiment, target_value, probability = predict_sentiment(model, tokenizer, new_text)

print(f"Predicted Sentiment: {sentiment}")
print(f"Predicted Target Value: {target_value}")
print(f"Probability: {probability}")

1/1 [==============================] - 0s 149ms/step
Predicted Sentiment: negative
Predicted Target Value: -1
Probability: [3.5566947e-18]


In [ ]:
# Save the model to the Colab file system
model.save("sentiment_model.h5")

# Save the tokenizer
with open("tokenizer.pkl", "wb") as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

# Download the saved files to your local machine
from google.colab import files

files.download("sentiment_model.h5")
files.download("tokenizer.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from keras.models import load_model
import pickle
from keras.preprocessing.sequence import pad_sequences

# Load the saved model
loaded_model = load_model("/content/drive/MyDrive/yt/sentiment_model.h5")

# Load the tokenizer
with open("/content/drive/MyDrive/yt/tokenizer.pkl", "rb") as tokenizer_file:
    loaded_tokenizer = pickle.load(tokenizer_file)

def predict_sentiment(model, tokenizer, text):
    # Tokenize and pad the input text
    sequences = tokenizer.texts_to_sequences([text])
    padded_sequences = pad_sequences(sequences, maxlen=model.input_shape[1])

    # Make predictions
    probability = model.predict(padded_sequences)[0]

    # Convert probability to sentiment label
    sentiment_label = "positive" if probability > 0.5 else "negative"

    # Convert sentiment label to target value
    target_value = 1 if sentiment_label == "positive" else -1

    return sentiment_label, target_value, probability

# Example usage:
new_text = "nice video"
sentiment, target_value, probability = predict_sentiment(loaded_model, loaded_tokenizer, new_text)

print(f"Predicted Sentiment: {sentiment}")
print(f"Predicted Target Value: {target_value}")
print(f"Probability: {probability}")

1/1 [==============================] - 0s 386ms/step
Predicted Sentiment: positive
Predicted Target Value: 1
Probability: [0.98200536]
